# 3D Mohr Plots

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import mplstereonet
import seaborn as sns
sns.set_style("white")
%matplotlib inline

In [ ]:
%load_ext watermark
print('author:')
%watermark -a "Irene Wallis" -d
print('\n')
print('requirements:')
%watermark -v -p pandas,matplotlib,mplstereonet,seaborn,watermark,fractoolbox 
print('\n')
%watermark -g